![This is the image of plant. ](https://d2r55xnwy6nx47.cloudfront.net/uploads/2020/07/Photosynthesis_2880_Lede.jpg)

The purpose of this notebook is to predict wheather the image belongs to diseased class or not. In this dataset we have 4 classes diseased leaf , diseased plant , freash leaf and freash plant.I will use basic CNN method to train the model and will check that testing image is belongs to which class.   

## Table of Content
1. Libraries
2. Training & DataLoader
3. Building the CNN
4. Summary
5. Training The CNN
6. Testing 
7. Conclusion

## Libraries

In [36]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import cv2

 ## Training & DataLoader

In [37]:
IMG_HEIGHT = 64
IMG_WIDTH = 64

train_datagen = ImageDataGenerator(rescale = 1./255,
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2,
    zoom_range=0.2, horizontal_flip=True,
    fill_mode="nearest")

In [38]:
training_set = train_datagen.flow_from_directory('../input/cotton-disease-dataset/Cotton Disease/train',
                                                 color_mode = "rgb",
                                                 shuffle = True,
                                                 seed = 42,
                                                 target_size=  (IMG_HEIGHT, IMG_WIDTH),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')

# Preprocessing the val set
val_datagen = ImageDataGenerator(rescale = 1./255)
val_set = val_datagen.flow_from_directory('../input/cotton-disease-dataset/Cotton Disease/val',
                                            color_mode = "rgb",
                                            shuffle = True,
                                            seed =42,
                                            target_size = (IMG_HEIGHT, IMG_WIDTH),
                                            batch_size = 64,
                                            class_mode = 'categorical')

Found 1951 images belonging to 4 classes.
Found 253 images belonging to 4 classes.


## Building the CNN

In [39]:
# Initialising the CNN
#cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
#cnn.add(tf.keras.layers.Conv2D(filters=32,padding="same",kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# Step 2 - Pooling
#cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
#cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',kernel_size=3, activation='relu'))
#cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
#cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',kernel_size=3, activation='relu'))
#cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
#cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
#cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
#cnn.add(tf.keras.layers.Dense(units=4, activation='softmax')) 

## Building the VGG-16

In [40]:


# Create a VGG-16 model
#vgg16 = tf.keras.applications.vgg16.VGG16(weights='imagenet')
#from tensorflow.keras.applications.resnet50 import ResNet50
#base_model = vgg16
#model = tf.keras.models.Sequential()
#for layer in base_model.layers[:]:
#    layer.trainable=True
    
    
#model.add(base_model)
#model.add(tf.keras.layers.Flatten())
#model.add(tf.keras.layers.Dense(512,kernel_regularizer=tf.keras.regularizers.l2(0.001), activation='relu', name='hidden1'))
#model.add(tf.keras.layers.Dropout(0.4))
#model.add(tf.keras.layers.Dense(256, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001), name='hidden2'))
#model.add(tf.keras.layers.Dropout(0.4))
#model.add(tf.keras.layers.Dense(4, activation='softmax', name='predictions'))


## Building the ResNet50

In [41]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT,IMG_WIDTH,3))
for layer in restnet.layers:
    layer.trainable = False
restnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [42]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
sgd = tf.keras.optimizers.SGD(lr=0.001, momentum=0.9)
model = Sequential()
model.add(restnet)
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(4, activation='softmax', name='predictions'))

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2, 2, 2048)        23587712  
_________________________________________________________________
dense_4 (Dense)              (None, 2, 2, 512)         1049088   
_________________________________________________________________
dropout_4 (Dropout)          (None, 2, 2, 512)         0         
_________________________________________________________________
dense_5 (Dense)              (None, 2, 2, 512)         262656    
_________________________________________________________________
dropout_5 (Dropout)          (None, 2, 2, 512)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 2048)              0         
_________________________________________________________________
predictions (Dense)          (None, 4)                

## Summary

In [43]:
#cnn.summary()


In [44]:
#model.summary()

## Training the CNN
## Fit the model
* get a history object
* If you see that `val_los` parameter is increasing that is *overfitting*. It happens when your model explains the training data too well, rather than picking up patterns that can help generalize over unseen data.

In [45]:
# Compiling the CNN
#cnn.compile(optimizer = 'adam',loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Training the CNN on the Training set and evaluating it on the Test set
#m = cnn.fit_generator(training_set, validation_data = val_set, epochs = 30)

In [ ]:
# Compiling the vgg-16
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
#sgd = tf.keras.optimizers.SGD(lr=0.001, momentum=0.9)
rms=tf.keras.optimizers.RMSprop()

#model.compile(loss='categorical_crossentropy',optimizer= sgd ,metrics=['accuracy'])

# Train the model
m=model.fit_generator(training_set, epochs=40,validation_data = val_set)


Epoch 1/40
31/31 [==============================] - 17s 543ms/step - loss: 1.4012 - accuracy: 0.3516 - val_loss: 1.3802 - val_accuracy: 0.3083
Epoch 2/40
31/31 [==============================] - 14s 465ms/step - loss: 1.3494 - accuracy: 0.3752 - val_loss: 1.4345 - val_accuracy: 0.3083
Epoch 3/40
31/31 [==============================] - 15s 500ms/step - loss: 1.3358 - accuracy: 0.4029 - val_loss: 1.3727 - val_accuracy: 0.3083
Epoch 4/40
31/31 [==============================] - 15s 471ms/step - loss: 1.3241 - accuracy: 0.4039 - val_loss: 1.3673 - val_accuracy: 0.3083
Epoch 5/40
31/31 [==============================] - 16s 510ms/step - loss: 1.3171 - accuracy: 0.4136 - val_loss: 1.3789 - val_accuracy: 0.3083
Epoch 6/40
28/31 [==========================>...] - ETA: 1s - loss: 1.3230 - accuracy: 0.4048

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15,5))
axs[0].plot(m.history['accuracy'])
axs[0].plot(m.history['val_accuracy'])
axs[0].set_title('Model accuracy')
axs[0].legend(['Train', 'Val'], loc='upper left')

axs[1].plot(m.history['loss'])
axs[1].plot(m.history['val_loss'])
axs[1].set_title('Model loss')
axs[1].legend(['Train', 'Val'], loc='upper left')

for ax in axs.flat:
    ax.set(xlabel='Epoch')

## Testing 

In [ ]:
# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = val_datagen.flow_from_directory('../input/cotton-disease-dataset/Cotton Disease/test',
                                            color_mode = "rgb",
                                            shuffle = True,
                                            seed =42,
                                            target_size = (IMG_HEIGHT, IMG_WIDTH),
                                            #batch_size = 32,
                                            class_mode = 'categorical')


In [ ]:

#print("[INFO] Calculating model accuracy")
#scores = cnn.evaluate(test_set)
#print(f"Test Accuracy: {scores[1]}")

print("[INFO] Calculating model accuracy")
scores = model.evaluate(test_set)
print(f"Test Accuracy: {scores[1]}")

##  Save the model
* Save weights to reuse them instead of training again. Keras function `save` creates h5 file with weights. Use `new_model.load_weights('model_cotton.h5')` to reuse it in other models.

In [ ]:
model.save('model_cotton.h5')

## Conclusion

We have tested the Freash leaf image from the testing dataset and our model is able to recognize it. 

## Please upvote this notebook if you find it useful.It really motivate to make new notebook.